In [1]:
# This notebook scrapes a popular book website for 'blurbs' by iterating through the titles
# in the books table from book-crossings

import numpy as np
import pandas as pd

import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
#BookCrossing
df_books = pd.read_csv('/Data Science/BX-CSV-Dump/BX-Books.csv', sep=';', error_bad_lines=False, encoding = "latin-1")

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:30

The website I am scraping has a consistent URL pattern that I exploit so that I can iterate through my dataset of books. 'starts' is a list of search query URLs for each book, and I feed this list into my scraper.

In [4]:
to_scrape = list(df_books.ISBN)
starts = []
for isbn in to_scrape:
    starts.append('https://www.goodreads.com/search?q=' + isbn)

Along with scraping the text of the 'blurb', I scrape the final URL. I cannot simply use the list of books in the dataset and line it up with scraped results due to heavy loss in the iteration of this algorithm. By scraping the URL along with blurb I will be able to match up the scraped blurbs with the right books.

In [5]:
import scrapy
from scrapy.crawler import CrawlerProcess

class ESSpider(scrapy.Spider):
    name = "ESS"
    start_urls = starts

    def parse(self, response):
        for article in response.xpath('//html'):
            
                yield {
                    'text': article.xpath('body/div[@class="content"]/div[@class="mainContentContainer "]/div[@class="mainContent "]/div[@class="mainContentFloat "]/div[@class="leftContainer"]/div[@id="topcol"]/div[@id="metacol"]/div[@id="descriptionContainer"]//span/text()').extract(),
                    'URL' : response.url,
                }
process = CrawlerProcess({
    'FEED_FORMAT': 'csv',
    'FEED_URI': 'blurbs2.csv', 
    'LOG_ENABLED': False,   
    'ROBOTSTXT_OBEY': True,
    'USER_AGENT': 'Joe Dobrow jdobrow@gmail.com',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True,
    'DOWNLOAD_DELAY' : .25
})

process.crawl(ESSpider)
process.start()
